In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import backend
import scipy as sc
from scipy import stats
import matplotlib.pyplot as plt
from itertools import product
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings; warnings.simplefilter('ignore')

# Black-Scholes price prediction 
The formula for the Black-Scholes Merton is given by:

$$
BS = S*\mathbb{N}(d_1) - K*\mathbb{N}(d_2),
$$

where 

$$
d_1 = (\log{(S/K)} + (r + \sigma^2)/2 )*T / (\sigma*\sqrt{T}), \\
d_2 = d1 - \sigma*\sqrt{T}
$$

- $S$ : Stock price
- $K$ : Strike
- $r$ : interest
- $T$ : Time to maturity
- $\mathbb{N}$: cdf of normal distribution

We will generate for these parameters random variables to compute different price values and we will use a linear regression model to predict price values and a deep neural network.




In [2]:
def Black_Scholes(K, S,r, sigma, T):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T)                                        )
    d2 = d1 - sigma*np.sqrt(T)
    price = S*sc.stats.norm.cdf(d1, 0,1) - K*sc.stats.norm.cdf(d2, 0,1)
    return price 

In [3]:
random_state = 42
np.random.seed(random_state)

S = np.arange(40, 61)
K = np.arange(20, 91)
r = np.arange(0, 0.051, 0.01)
T = np.arange(3/12, 2.01, 1/12)
sigma = np.arange(0.1, 0.81, 0.1)

option_prices = pd.DataFrame(
  product(S, K, r, T, sigma),
  columns=["S", "K", "r", "T", "sigma"]
)

option_prices["Call"] = Black_Scholes(
  option_prices["S"].values, 
  option_prices["K"].values, 
  option_prices["r"].values, 
  option_prices["T"].values, 
  option_prices["sigma"].values
)

option_prices = (option_prices
  .assign(
    observed_price=lambda x: (
      x["Call"] + np.random.normal(scale=0.15)
    )
  )
)

In [4]:
option_prices

,S,K,r,T,sigma,Call,observed_price
0,40,20,0.00,0.25,0.1,2.266781e-19,0.074507
1,40,20,0.00,0.25,0.2,1.373904e-10,0.074507
2,40,20,0.00,0.25,0.3,1.478773e-07,0.074507
3,40,20,0.00,0.25,0.4,5.431553e-06,0.074513
4,40,20,0.00,0.25,0.5,5.037565e-05,0.074557
...,...,...,...,...,...,...,...
1574491,60,90,0.05,2.00,0.4,5.201661e+01,52.091118
1574492,60,90,0.05,2.00,0.5,5.535526e+01,55.429770
1574493,60,90,0.05,2.00,0.6,5.825035e+01,58.324855
1574494,60,90,0.05,2.00,0.7,6.079729e+01,60.871800


In [5]:
option_prices.shape

(1574496, 7)

In [6]:
train_data, test_data = train_test_split(
  option_prices, 
  test_size=0.01, random_state=random_state
)

In [7]:
train_data.shape

(1558751, 7)

In [8]:
test_data.shape

(15745, 7)

In [9]:
X_train = train_data.drop(['Call', 'observed_price'], axis = 1).values
y_train = train_data['observed_price'].values

In [10]:
y_train

array([23.73744518,  0.74426706, 29.85556666, ..., 40.10712747,
       41.96504722, 42.79765332])

In [11]:
scaler = StandardScaler().fit(X_train)
X_train = np.round(scaler.transform(X_train),2)

In [12]:
X_train

array([[-1.49, -0.54,  1.46,  0.87,  1.53],
       [ 0.33, -1.12,  0.88, -1.34,  1.53],
       [-1.49,  0.73, -0.29, -1.18,  0.65],
       ...,
       [-1.49,  0.88,  1.46,  0.39,  0.22],
       [-0.33,  1.56,  0.29, -0.39, -0.22],
       [-1.49,  0.44, -0.29,  1.5 ,  1.09]])

In [13]:
y_train = np.round(y_train,2)
y_train

array([23.74,  0.74, 29.86, ..., 40.11, 41.97, 42.8 ])

In [14]:
X_test= test_data.drop(['Call', 'observed_price'], axis = 1).values
y_test= test_data['observed_price'].values
scaler = StandardScaler().fit(X_test)
X_test = np.round(scaler.transform(X_test),2)

In [15]:
X_test

array([[ 0.67,  0.58, -1.48,  1.05,  1.52],
       [-0.98, -1.13,  0.87, -1.33,  1.52],
       [ 1.17,  0.44, -0.89,  1.68,  1.52],
       ...,
       [-1.64,  1.51, -1.48, -0.38, -0.23],
       [ 1.33, -1.37, -0.89,  1.36,  1.52],
       [-0.16, -1.03, -0.89,  1.36,  0.65]])

In [16]:
model = LinearRegression()

In [17]:
model.fit(X_train, y_train)

LinearRegression()

In [18]:
model.coef_.round(decimals=4)

array([-2.44050e+00,  1.35793e+01, -2.50000e-03,  5.14190e+00,
        3.16130e+00])

In [19]:
model.intercept_.round(decimals=4)

20.0123

In [20]:
y_test_hat = np.round(model.predict(X_test),2)

#predicted price on the test data
y_test_hat

array([36.46,  5.02, 36.58, ..., 41.84,  9.96, 15.47])

In [21]:
model.score(X_test, y_test)

0.9453301367552494

In [22]:
mean_squared_error(y_test, y_test_hat)

13.063405461037322

In [23]:
 r2_score(y_test, y_test_hat)

0.9453292920853239

# Neural networks

In [24]:
'''

Creating a simple neural network to predict
the call option price from the Black-Scholes formula

'''

NN = Sequential()
NN.add(Dense(120, input_dim = X_train.shape[1]))
NN.add(Dense(120, activation='relu'))
NN.add(Dense(1))
NN.compile(loss='mse', optimizer='rmsprop')

In [25]:
NN.fit(X_train, y_train,batch_size=64, epochs=10)
results = NN.evaluate(X_test, y_test)

Epoch 1/10

24356/24356 [==============================] - 76s 3ms/step - loss: 1.1903
Epoch 2/10
24356/24356 [==============================] - 77s 3ms/step - loss: 0.0900
Epoch 3/10
24356/24356 [==============================] - 80s 3ms/step - loss: 0.0660
Epoch 4/10
24356/24356 [==============================] - 87s 4ms/step - loss: 0.0541
Epoch 5/10
24356/24356 [==============================] - 82s 3ms/step - loss: 0.0459
Epoch 6/10
24356/24356 [==============================] - 80s 3ms/step - loss: 0.0401
Epoch 7/10
24356/24356 [==============================] - 75s 3ms/step - loss: 0.0361
Epoch 8/10
24356/24356 [==============================] - 80s 3ms/step - loss: 0.0322
Epoch 9/10
24356/24356 [==============================] - 79s 3ms/step - loss: 0.0294
Epoch 10/10
493/493 [==============================] - 2s 3ms/step - loss: 0.0114


In [26]:
results

0.011357421055436134

In [27]:
y_train_hat = NN.predict(X_train)

48711/48711 [==============================] - 108s 2ms/step


In [28]:
# predicted call price
y_train_hat

array([[23.627998 ],
       [ 0.6995739],
       [29.883297 ],
       ...,
       [40.1602   ],
       [41.964745 ],
       [42.863903 ]], dtype=float32)